## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBSS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
393
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+-----------+------+--------------------+------+
|11183.467759022718|2476.4223902765893| 385.1362997238969|   2493.3|SSSSBBSS| 33.9833|  -109.75|       SNWD|0.09903383540055485|0.05733362444014835|0.05647398712192465|0.969514518488140

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00023926,1979.0,2476.422390
1,USC00023926,1982.0,-32.223274
2,USC00023926,1983.0,1268.582582
3,USC00023926,1973.0,2159.949825
4,USC00028326,1973.0,658.551995


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00020159,USC00020170,USC00020174,USC00020855,USC00021749,USC00022779,USC00023082,USC00023448,USC00023505,USC00023683,...,USC00028162,USC00028326,USC00029271,USC00029367,USC00029622,USC00294375,USC00295273,USC00297191,USC00297386,USW00093139
year,,,,,,,,,,,,,,,,,,,,,
1905.0,NaN,NaN,NaN,-142.697762,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-251.433336,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,202.923926,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-31.346297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,-22.381274,NaN,NaN,-76.752740,NaN,NaN,NaN,NaN,NaN,565.602483,...,-4.028565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,-118.971758,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-6.695847,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-46.159441,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'snow'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

snow
total RMS                   =  705.379279328
RMS removing mean-by-station=  643.818287689
RMS removing mean-by-year   =  501.937985779


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    #print T.shape
    mean_by_year=np.nanmean(T,axis=1)
    #print mean_by_year
    T=(T.transpose()-mean_by_year).transpose()   
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 705.379279328
0 after removing mean by year    = 501.937985779
0 after removing mean by stations= 463.630388244
1 after removing mean by year    = 456.261858874
1 after removing mean by stations= 454.069976974
2 after removing mean by year    = 453.244063281
2 after removing mean by stations= 452.885670348
3 after removing mean by year    = 452.714107247
3 after removing mean by stations= 452.626129563
4 after removing mean by year    = 452.578900502
4 after removing mean by stations= 452.552799563


In [10]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00023926,1979.0,11183.467759
1,USC00023926,1982.0,7613.756556
2,USC00023926,1983.0,9361.092523
3,USC00023926,1973.0,11159.371711
4,USC00028326,1973.0,11602.290222


In [11]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00020159,USC00020170,USC00020174,USC00020855,USC00021749,USC00022779,USC00023082,USC00023448,USC00023505,USC00023683,...,USC00028162,USC00028326,USC00029271,USC00029367,USC00029622,USC00294375,USC00295273,USC00297191,USC00297386,USW00093139
year,,,,,,,,,,,,,,,,,,,,,
1905.0,NaN,NaN,NaN,-54.716925,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-102.261873,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,56.299602,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-121.758574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,1686.979346,NaN,NaN,-58.859091,NaN,NaN,NaN,NaN,NaN,4152.240374,...,-93.152845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,-51.163824,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-121.867459,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-159.310355,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'SNOW-Coeff1'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

SNOW-Coeff1
total RMS                   =  1814.35230759
RMS removing mean-by-station=  955.26536178
RMS removing mean-by-year   =  1397.53019121


In [13]:
T=year_station_table
print("Coeff 1")
print 'initial RMS=',RMS(T)

print "PRCP"
for i in range(5):
    #print T.shape
    mean_by_year=np.nanmean(T,axis=1)
    #print mean_by_year
    T=(T.transpose()-mean_by_year).transpose()   
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

Coeff 1
initial RMS= 1814.35230759
PRCP
0 after removing mean by year    = 1397.53019121
0 after removing mean by stations= 890.088128258
1 after removing mean by year    = 754.937603852
1 after removing mean by stations= 711.021815652
2 after removing mean by year    = 695.198755218
2 after removing mean by stations= 688.834205833
3 after removing mean by year    = 685.979905012
3 after removing mean by stations= 684.579789203
4 after removing mean by year    = 683.847713249
4 after removing mean by stations= 683.448851453


In [14]:
#extract longitude and latitude for each station
feature='coeff_2'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00023926,1979.0,2476.422390
1,USC00023926,1982.0,-32.223274
2,USC00023926,1983.0,1268.582582
3,USC00023926,1973.0,2159.949825
4,USC00028326,1973.0,658.551995


In [15]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00020159,USC00020170,USC00020174,USC00020855,USC00021749,USC00022779,USC00023082,USC00023448,USC00023505,USC00023683,...,USC00028162,USC00028326,USC00029271,USC00029367,USC00029622,USC00294375,USC00295273,USC00297191,USC00297386,USW00093139
year,,,,,,,,,,,,,,,,,,,,,
1905.0,NaN,NaN,NaN,-142.697762,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-251.433336,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,202.923926,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-31.346297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,-22.381274,NaN,NaN,-76.752740,NaN,NaN,NaN,NaN,NaN,565.602483,...,-4.028565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,-118.971758,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-6.695847,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-46.159441,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'SNWD-Coeff2'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

SNWD-Coeff2
total RMS                   =  705.379279328
RMS removing mean-by-station=  643.818287689
RMS removing mean-by-year   =  501.937985779


In [17]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


,station,year,coeff_3
0,USC00023926,1979.0,385.136300
1,USC00023926,1982.0,-689.642002
2,USC00023926,1983.0,-1692.249084
3,USC00023926,1973.0,-378.367818
4,USC00028326,1973.0,1898.158307


In [18]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station,USC00020159,USC00020170,USC00020174,USC00020855,USC00021749,USC00022779,USC00023082,USC00023448,USC00023505,USC00023683,...,USC00028162,USC00028326,USC00029271,USC00029367,USC00029622,USC00294375,USC00295273,USC00297191,USC00297386,USW00093139
year,,,,,,,,,,,,,,,,,,,,,
1905.0,NaN,NaN,NaN,-122.892273,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-328.131300,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,-154.523119,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-72.744722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915.0,102.714449,NaN,NaN,-55.939499,NaN,NaN,NaN,NaN,NaN,-401.793166,...,106.325439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1916.0,NaN,NaN,NaN,42.990105,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,88.411975,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-61.029943,NaN,NaN,NaN,NaN,NaN
1934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'SNWD-Coeff3'

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

SNWD-Coeff3
total RMS                   =  576.182922986
RMS removing mean-by-station=  546.041731496
RMS removing mean-by-year   =  402.25928699
